Custom CNN на CIFAR-100

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar100

# Загрузка данных CIFAR-100
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

# Нормализация данных
x_train, x_test = x_train / 255.0, x_test / 255.0

# Преобразование меток в one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, 100)
y_test = tf.keras.utils.to_categorical(y_test, 100)

# Определение кастомной CNN
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(100, activation='softmax')  # 100 классов для CIFAR-100
])

# Компиляция модели
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Обучение модели
history = model.fit(x_train, y_train, epochs=20,
                    validation_data=(x_test, y_test),
                    batch_size=64)


169001437/169001437 ━━━━━━━━━━━━━━━━━━━━ 10s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 70s 87ms/step - accuracy: 0.0590 - loss: 4.2170 - val_accuracy: 0.1764 - val_loss: 3.4426
Epoch 2/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 66s 84ms/step - accuracy: 0.2044 - loss: 3.2926 - val_accuracy: 0.2578 - val_loss: 3.0278
Epoch 3/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 82s 84ms/step - accuracy: 0.2797 - loss: 2.8944 - val_accuracy: 0.2975 - val_loss: 2.8459
Epoch 4/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 82s 84ms/step - accuracy: 0.3321 - loss: 2.6361 - val_accuracy: 0.3345 - val_loss: 2.6663
Epoch 5/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 82s 84ms/step - accuracy: 0.3741 - loss: 2.4364 - val_accuracy: 0.3494 - val_loss: 2.5781
Epoch 6/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 82s 84ms/step - accuracy: 0.4065 - loss: 2.2826 - val_accuracy: 0.3712 - val_loss: 2.5002
Epoch 7/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 69s 88ms/step - accuracy: 0.4367 - loss: 2.1428 - val_accuracy: 0.3652 - val_loss: 2.5173
Epoch 8/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 67s 86ms/step - accuracy: 0.4601 - loss: 2.0286 - 

Fine-tuning ResNet-50 на CIFAR-100

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.optimizers import Adam

# Загрузка данных CIFAR-100
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

# Нормализация данных
x_train, x_test = x_train / 255.0, x_test / 255.0

# Преобразование меток в one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, 100)
y_test = tf.keras.utils.to_categorical(y_test, 100)

# Загрузка ResNet50 с предобученными весами ImageNet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Заморозка базовой модели
base_model.trainable = False

# Добавление кастомного классификатора
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dense(100, activation='softmax')  # 100 классов для CIFAR-100
])

# Компиляция модели
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Обучение модели
history = model.fit(x_train, y_train, epochs=20,
                    validation_data=(x_test, y_test),
                    batch_size=64)

# Разморозка части слоев и дообучение
base_model.trainable = True

# Компиляция модели с меньшим learning_rate для fine-tuning
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Продолжение обучения
history_fine = model.fit(x_train, y_train, epochs=10,
                         validation_data=(x_test, y_test),
                         batch_size=64)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Epoch 1/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 223s 275ms/step - accuracy: 0.0134 - loss: 4.6742 - val_accuracy: 0.0227 - val_loss: 4.5344
Epoch 2/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 265s 279ms/step - accuracy: 0.0285 - loss: 4.5184 - val_accuracy: 0.0411 - val_loss: 4.4616
Epoch 3/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 252s 267ms/step - accuracy: 0.0394 - loss: 4.4428 - val_accuracy: 0.0465 - val_loss: 4.4012
Epoch 4/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 265s 270ms/step - accuracy: 0.0523 - loss: 4.3809 - val_accuracy: 0.0501 - val_loss: 4.3474
Epoch 5/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 268s 279ms/step - accuracy: 0.0610 - loss: 4.3279 - val_accuracy: 0.0618 - val_loss: 4.2992
Epoch 6/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 255s 270ms/step - accuracy: 0.0683 - loss: 4.2795 - val_accuracy: 0.0638 - val_loss: 4.2610
Epoch 7/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 268s 278ms/step - accuracy: 0.0718 - loss: 4.2424 - val_accuracy: 0.0660 - val_loss: 4.2320
Epoch 8/20
782/782 ━━━━━━

Fine-tuning ResNet-50 с аугментацией данных

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.optimizers import Adam

# Загрузка данных CIFAR-100
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

# Нормализация данных
x_train, x_test = x_train / 255.0, x_test / 255.0

# Преобразование меток в one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, 100)
y_test = tf.keras.utils.to_categorical(y_test, 100)

# Аугментация данных
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

datagen.fit(x_train)

# Загрузка ResNet50 с предобученными весами ImageNet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Заморозка базовой модели
base_model.trainable = False

# Добавление кастомного классификатора
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dense(100, activation='softmax')  # 100 классов для CIFAR-100
])

# Компиляция модели
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Обучение модели с использованием аугментации данных
history = model.fit(datagen.flow(x_train, y_train, batch_size=64),
                    epochs=20,
                    validation_data=(x_test, y_test))

# Разморозка части слоев и дообучение
base_model.trainable = True

# Компиляция модели с меньшим learning_rate для fine-tuning
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Продолжение обучения
history_fine = model.fit(datagen.flow(x_train, y_train, batch_size=64),
                         epochs=10,
                         validation_data=(x_test, y_test))


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


782/782 ━━━━━━━━━━━━━━━━━━━━ 346s 430ms/step - accuracy: 0.0129 - loss: 4.6683 - val_accuracy: 0.0221 - val_loss: 4.5539
Epoch 2/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 338s 432ms/step - accuracy: 0.0220 - loss: 4.5566 - val_accuracy: 0.0364 - val_loss: 4.4927
Epoch 3/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 328s 419ms/step - accuracy: 0.0282 - loss: 4.5117 - val_accuracy: 0.0405 - val_loss: 4.4554
Epoch 4/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 385s 423ms/step - accuracy: 0.0337 - loss: 4.4751 - val_accuracy: 0.0489 - val_loss: 4.4076
Epoch 5/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 377s 417ms/step - accuracy: 0.0371 - loss: 4.4503 - val_accuracy: 0.0521 - val_loss: 4.3819
Epoch 6/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 395s 433ms/step - accuracy: 0.0413 - loss: 4.4180 - val_accuracy: 0.0600 - val_loss: 4.3401
Epoch 7/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 333s 370ms/step - accuracy: 0.0455 - loss: 4.3976 - val_accuracy: 0.0587 - val_loss: 4.3167
Epoch 8/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 329s 380ms/step - accuracy: 0.0491 - loss: 4.37

Custom CNN: Это базовая модель CNN, специально разработанная для CIFAR-100.

ResNet-50: Здесь используется предобученная модель с ImageNet, заменены финальные слои для обучения на CIFAR-100.

ResNet-50 с аугментацией: Добавлена аугментация данных для улучшения генерализации модели.